## IMPORT LIBRARIES

In [1]:
import numpy
import pandas as pd
from tensorflow import keras
layers = keras.layers
models = keras.models
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


# LOADING DATA SET

In [2]:
dataframe = pd.read_csv('sonar.csv',header = None)
dataset = dataframe.values
dataset

array([[0.02, 0.0371, 0.0428, ..., 0.009, 0.0032, 'R'],
       [0.0453, 0.0523, 0.0843, ..., 0.0052, 0.0044, 'R'],
       [0.0262, 0.0582, 0.1099, ..., 0.0095, 0.0078, 'R'],
       ...,
       [0.0522, 0.0437, 0.018, ..., 0.0077, 0.0031, 'M'],
       [0.0303, 0.0353, 0.049, ..., 0.0036, 0.0048, 'M'],
       [0.026, 0.0363, 0.0136, ..., 0.0061, 0.0115, 'M']], dtype=object)

## SPLIT DATA INTO INPUT & OUTPUT VARIABLES

In [3]:
# split into input (X) and output (Y) variables
x = dataset[:,0:60].astype(float) 
y = dataset[:,60]
x.shape
y.shape
#y.shape

(208,)

## LABEL ENCODER OF OUTPUT

In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y)
y_train = encoder.transform(y)
y_train


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## CREATE BASELINE MODEL

In [5]:
def create_baseline(): 
    network = models.Sequential()
    network.add(layers.Dense(60, activation='relu', input_shape=(60,)))
    network.add(layers.Dense(1, activation='sigmoid'))
    network.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return network
    
    

## EVALUATE BASELINE MODEL 

In [6]:
# evaluate model with standardized dataset
seed = 7
numpy.random.seed(seed)
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x,y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Results: 85.11% (6.19%)


## EVALUATE BASELINE MODEL WITH STANDARDIZED DATASET

In [6]:
seed = 7
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, x, y_train, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 85.57% (6.40%)


## WITH SMALL MODEL

In [5]:
def create_baseline(): 
    network = models.Sequential()
    network.add(layers.Dense(30, activation='relu', input_shape=(60,)))
    network.add(layers.Dense(1, activation='sigmoid'))
    network.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return network

seed = 7
numpy.random.seed(seed)
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x,y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Results: 83.16% (6.22%)


## WITH LARGE MODEL

In [5]:
def create_baseline(): 
    network = models.Sequential()
    network.add(layers.Dense(60, activation='relu', input_shape=(60,)))
    network.add(layers.Dense(60, activation='relu'))
    network.add(layers.Dense(1, activation='sigmoid'))
    network.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return network

seed = 7
numpy.random.seed(seed)
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x,y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 85.09% (5.86%)


## WITH MORE EPOCHS

In [6]:
def create_baseline(): 
    network = models.Sequential()
    network.add(layers.Dense(60, activation='relu', input_shape=(60,)))
    network.add(layers.Dense(1, activation='sigmoid'))
    network.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return network

seed = 7
numpy.random.seed(seed)
estimator = KerasClassifier(build_fn=create_baseline, epochs=200, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x,y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 83.66% (5.72%)
